In [14]:
from ECKG.src.baseline import BaselineController, MentionPairFeatures
from contextual_model_bert import ContextualControllerBERT
from data import read_classla, read_corpus
from eventify import *
from helper_functions import *

import urllib.request

sl_text  = "Ljubljana - Evropska komisija je Sloveniji za letos napovedala 1,9 - odstotno rast , za naslednje leto pa 2,5 - odstotno , kar je nad povprečjem v območju evra in EU . Sicer pa bo Slovenija letos po napovedih iz Bruslja beležila 5,8 - odstotni javnofinančni primanjkljaj , naslednje leto pa petodstotnega , kar je slabše od povprečja v območju evra in EU . Javni dolg Slovenije se bo po napovedih komisije vztrajno večal , a bo še vedno bistveno pod povprečjem območja evra in EU ter bistveno pod referenčno mejo 60 odstotkov bruto domačega proizvoda ( BDP ) , ki jo določa pakt za stabilnost in rast . Za letos komisija Sloveniji napoveduje 42,8 - odstotni dolg , za prihodnje leto pa 46 - odstotnega . Poleg tega Evropska komisija Sloveniji za letos napoveduje 2,6 - odstotno inflacijo , za prihodnje leto pa 2,1 - odstotno . Inflacija v Sloveniji bo tako letos po napovedih komisije enaka kot v povprečju v območju evra in pod povprečjem EU , prihodnje leto pa nekoliko nad povprečjem v območju evra in celotne EU . Po napovedih iz Bruslja bo Slovenija letos beležila večjo brezposelnost kot lani . Medtem ko je bila ta lani po ocenah komisije 7,3 - odstotna , naj bi bila letos 8,2 - odstotna , prihodnje leto pa osemodstotna . To je še vedno manj kot v povprečju v območju evra in celotne EU , kjer naj bi bila letos 10 - odstotna , prihodnje leto pa 9,7 - odstotna oziroma 9,5 - odstotna in 9,1 - odstotna . V lanski novembrski gospodarski napovedi je komisija Sloveniji za letos prav tako napovedala 1,9 - odstotno rast , za leto 2012 pa 2,6 - odstotno , a njene napovedi za javnofinančni primanjkljaj Slovenije so bile novembra boljše kot danes . Novembra je namreč napovedala , da bo ta v Sloveniji letos 5,3 - odstoten , naslednje leto pa 4,7 - odstoten . Slovenija je sicer - kot velika večina držav EU - zaradi prekomernega javnofinančnega primanjkljaja v postopku Evropske komisije . Evropska komisija je kot rok za znižanje javnofinančnega primanjkljaja Slovenije pod tri odstotke BDP , kar kot zgornjo mejo določa pakt za stabilnost in rast , določila leto 2013 . Slovenija je po navedbah finančnega ministrstva v programu stabilnosti , ki ga je posredovala Evropski komisiji , zapisala , da za leto 2012 načrtuje znižanje primanjkljaja na 3,9 odstotka BDP . Na zvišanje letošnjega primanjkljaja bo sicer po navedbah komisije vplivala dokapitalizacija največje slovenske banke , Nove Ljubljanske banke . Glede primanjkljaja v letu 2012 je komisija zapisala , da je izračunan ob predpostavki , da ne bo dodatnih ukrepov . Glede nadaljnjih ukrepov za zajezitev izdatkov v letu 2012 sicer komisija ugotavlja , da ti v glavnem še niso podrobno opredeljeni in dogovorjeni . Glede gospodarske rasti pa Evropska komisija v primeru Slovenije ugotavlja razmeroma medlo okrevanje , ki temelji na izvozu . Ta pa je po njenih navedbah ohromljen zaradi izgub konkurenčnosti , ki so se nabrale pred in med krizo , ter zaradi usmerjenosti izvoza na gospodarsko šibek Zahodni Balkan . Drugi dejavnik , ki zavira gospodarsko rast Slovenije , so po navedbah Evropske komisije razmere v gradbeništvu . Poleg tega se je po navedbah komisije v prizadevanjih bank za izboljšanje svojih bilanc skrčil tudi tok posojil v realno gospodarstvo"


# Download text and decode
# Note: Set proxy if behind a proxy (https://docs.python.org/2/library/urllib.html)
url = "http://www.gutenberg.org/files/1342/1342-0.txt"
en_text = urllib.request.urlopen(url).read().decode()
en_text = ".".join(en_text.split(".")[20:50])



## Slovene Coreference resolution

In [18]:
model_name = "baseline_model_senticoref"
learning_rate = 0.05
dataset_name = "senticoref"
# BASELINE
model = BaselineController(MentionPairFeatures.num_features(),
                           model_name=model_name,
                           learning_rate=learning_rate,
                           dataset_name=dataset_name
                           )
model = BaselineController.from_pretrained("baseline_model/baseline_model_senticoref")

# CONTEXTUAL BERT
model = ContextualControllerBERT.from_pretrained("contextual_model_bert/fold7_0")

# read corpus
documents = read_corpus(dataset_name)

# test on first document
eval_doc = model._train_doc(documents[0], eval_mode=True)
print("ORIGINAL CLUSTERS")
for cluster in documents[0].clusters:
    print([" ".join([y.raw_text for y in documents[0].mentions[x].tokens]) for x in cluster])
print("PREDICTED CLUSTERS")
for key, values in eval_doc[0].items():
    if key is None:
        key = "None"
    try:
        print(" ".join([y.raw_text for y in documents[0].mentions[key].tokens]), ":")
        print([" ".join([y.raw_text for y in documents[0].mentions[x].tokens]) for x in values])
    except KeyError:
        continue
# Evaluation for conll format produced by classla
#     classla.download('sl')
#     pipeline = classla.Pipeline("sl", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)
#
#     # Analyze using classla/stanza to get Document format
#     data = pipeline(sl_text)
#     # Convert to conll format
#     data = data.to_conll()
#     # Convert to format used by slovene-coreference-resolution library (https://github.com/matejklemen/slovene-coreference-resolution)
#     # TODO: need to fix some mistakes when converting formats
#     data = read_classla(data)
    # res, _ = model._train_doc(data3, eval_mode=True)

INFO:root:Initialized baseline model with name baseline_model_senticoref
INFO:root:Initialized baseline model with name baseline_model_senticoref
Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ORIGINAL CLUSTERS
['Ljubljana']
['Evropska komisija', 'komisije', 'komisija', 'Evropska komisija', 'komisije', 'komisije', 'komisija', 'njene', 'napovedala', 'Evropske komisije', 'Evropska komisija', 'Evropski komisiji', 'komisija', 'komisija', 'Evropska komisija', 'njenih', 'Evropske komisije', 'komisije']
['Sloveniji', 'Slovenija', 'Slovenije', 'Sloveniji', 'Sloveniji', 'Sloveniji', 'Slovenija', 'Sloveniji', 'Slovenije', 'Sloveniji', 'Slovenija', 'Slovenije', 'Slovenija', 'posredovala', 'načrtuje', 'Slovenije', 'Slovenije']
['EU', 'EU', 'EU', 'EU', 'EU', 'EU', 'EU']
['Bruslja', 'Bruslja']
['največje slovenske banke', 'Nove Ljubljanske banke']
['Zahodni Balkan']
PREDICTED CLUSTERS
Sloveniji :
['Slovenija', 'Slovenije', 'Slovenija']
EU :
['EU', 'EU']
Evropska komisija :
['komisije', 'komisija', 'Evropska komisija']
Slovenija :
['Sloveniji', 'Sloveniji']
Sloveniji :
['Sloveniji', 'Slovenija', 'Slovenije']
komisije :
['komisije', 'komisija', 'njenih']
EU :
['EU']
EU :
['EU', 'Evropske ko

## NER + SVN triplet extraction

### Slovene

In [2]:
# Initialize Slovene classla pipeline
classla.download('sl')
sl_pipeline = classla.Pipeline("sl", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)

INFO:classla:Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/pos/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/lemma/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/depparse/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/ner/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/pretrain/standard.pt.
INFO:classla:Finished downloading models and saved to /home/erikkastelec/classla_resources.
INFO:classla:Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| deppar

In [4]:
# Analyze using classla/stanza to get Document format
data = sl_pipeline(sl_text)
print("extracted named entities".upper())
print([" ".join([y.text for y in x.tokens]) for x in data.entities])

# Run named entity deduplication/resolution
deduplication_mapper = deduplicate_named_entities(data)
print("deduplication mapper".upper())
print(deduplication_mapper)

# SVO triplet / event extraction
e = Eventify(language="sl")
# Extract SVO triplets / events
events = e.eventify(sl_text)
print("SVO triplets / events".upper())
print(events)
#TODO: Replace with coreferences and map with deduplication mapper

EXTRACTED NAMED ENTITIES
['Ljubljana', 'Evropska komisija', 'Sloveniji', 'EU', 'Slovenija', 'Bruslja', 'EU', 'Slovenije', 'EU', 'Sloveniji', 'Evropska komisija Sloveniji', 'Sloveniji', 'EU', 'EU', 'Bruslja', 'Slovenija', 'EU', 'Sloveniji', 'Slovenije', 'Sloveniji', 'Slovenija', 'EU', 'Evropske komisije', 'Evropska komisija', 'Slovenije', 'Slovenija', 'Evropski komisiji', 'Nove Ljubljanske banke', 'Evropska komisija', 'Slovenije', 'Zahodni Balkan', 'Slovenije', 'Evropske komisije razmere']
DEDUPLICATION MAPPER
{'Ljubljana': 'Ljubljana', 'Evropska komisija': 'Evropska komisija', 'Evropska komisija Sloveniji': 'Evropska komisija', 'Evropske komisije': 'Evropska komisija', 'Evropski komisiji': 'Evropska komisija', 'Evropske komisije razmere': 'Evropska komisija', 'Sloveniji': 'Sloveniji', 'Slovenija': 'Sloveniji', 'Slovenije': 'Sloveniji', 'EU': 'EU', 'Bruslja': 'Bruslja', 'Nove Ljubljanske banke': 'Nove Ljubljanske banke', 'Zahodni Balkan': 'Zahodni Balkan'}
SVO TRIPLETS / EVENTS
[('Evrop

### English

In [5]:
# Initialize English stanza pipeline
stanza.download("en")
en_pipeline = stanza.Pipeline("en", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)

INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /home/erikkastelec/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /home/erikkastelec/stanza_resources.


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [15]:
# Analyze using classla/stanza to get Document format
data = en_pipeline(en_text)
print("extracted named entities".upper())
print([" ".join([y.text for y in x.tokens]) for x in data.entities])

# Run named entity deduplication/resolution
deduplication_mapper = deduplicate_named_entities(data)
print("deduplication mapper".upper())
print(deduplication_mapper)

e = Eventify(language="en")
# Extract SVO triplets / events
events = e.eventify(en_text)
print("SVO triplets / events".upper())
print(events)
#TODO: Replace with coreferences and map with deduplication mapper

EXTRACTED NAMED ENTITIES
['Bingley', 'four or five thousand', 'Bennet', 'one', 'one', 'Bingley', 'five', 'Bingley', 'one', 'William', 'Lady Lucas', 'Bingley', 'Lizzy', 'Lizzy', 'Jane', 'Lydia', 'Lizzy', 'Bennet']
DEDUPLICATION MAPPER
{'Bingley': 'Bingley', 'four or five thousand': 'four or five thousand', 'Bennet': 'Bennet', 'one': 'one', 'five': 'five', 'William': 'William', 'Lady Lucas': 'Lady Lucas', 'Lizzy': 'Lizzy', 'Jane': 'Jane', 'Lydia': 'Lydia'}
SVO TRIPLETS / EVENTS
[('he', 'single', '<|EMPTY|>'), ('it', 'affect', 'them'), ('his wife', 'dear Mr. Bennet', '<|EMPTY|>'), ('his wife', 'reply', 'My dear Mr. Bennet'), ('his wife', 'reply', 'how you be so tiresome'), ('You', 'know', ('I', 'think', '<|EMPTY|>')), ('you', 'nonsense talk', '<|EMPTY|>'), ('he', 'fall', '<|EMPTY|>'), ('you', 'visit', 'him'), ('you', '', '<|EMPTY|>'), ('he', 'come', '<|EMPTY|>'), ('I', 'see', 'occasion for that'), ('You', 'go', '<|EMPTY|>'), ('girls', 'go', '<|EMPTY|>'), ('you', 'send', 'them'), ('Mr. Bin